## This notebook is for creating the "planned facilities" layer
---
The planned facilities are marked as funded or ones that were installed after 2016. Data come from the City of Atlanta.

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

from bikewaysim.paths import config, root

In [2]:
#TODO, resolve difference between the old inventory and the old as some streets had their facilities upgraded recently (Edgewood) and (Confederate/United)
coa = gpd.read_file(config['coa_bike_fp'])
coa.to_crs(config['projected_crs_epsg'],inplace=True)

In [3]:
# only keep funded ones or ones built post 2016
coa = coa[(coa['Status']=='Funded') | (coa['YearInstalled'] > 2016)]

In [4]:
# remove uncessary columns
coa.drop(columns=['GlobalID','Shape__Length','LengthMi','CrossSectionNotes','Status'],inplace=True)

#rename the id/year column
coa.rename(columns={'OBJECTID':'id'},inplace=True)

# rename columns for consistency
coa.columns = ['coa_'+col.lower() if col != 'geometry' else 'geometry' for col in coa.columns.tolist()]

# convert facility type to OSM (use highest protection if two different types)
osm_types = ['sharrow','bike lane','buffered bike lane','cycletrack','multi use path']
coa_conversion = {
       'Protected Bike Lane': osm_types[3], 
       'Protected Bike Lane / Bike Lane': osm_types[3],
       'Two-Way Cycle Track': osm_types[3], 
       'Uphill Bike Lane / Downhill Sharrows': osm_types[1],
       'Sharrows': osm_types[0], 
       'Bike Lane': osm_types[1], 
       'Bike Lane ': osm_types[1],
       'Uphill Buffered Bike Lane / Downhill Sharrows': osm_types[2],
       'Buffered Bike Lane': osm_types[2], 
       'Buffered Contraflow Bike Lane / Bike Lane': osm_types[1],
       'Shared-Use Path': osm_types[4], 
       'Neighborhood Greenway': osm_types[0], 
       'Bike Lane / Sharrows': osm_types[1],
       'Shared-Use Path / Bike Lane': osm_types[4], 
       'Buffered Bike Lane / Bike Lane': osm_types[2],
       'Buffered Bike Lane / Shared-Use Path': osm_types[4],
       'Shared-Use Path / Sharrows': osm_types[4],
       'Uphill Protected Bike Lane / Downhill Sharrows': osm_types[3], 
       'Shared Path': osm_types[4],
       'Seperated Bike Lane': osm_types[3],
       'Contraflow Bike Lane': osm_types[1], 
       'Bike Boulevard': osm_types[0],
       'Contraflow Bike Lane / Sharrows': osm_types[2], 
       'Separated Bike Lane / Buffered Bike Lane': osm_types[3],
       'Separated Bike Lane / Bike Lane': osm_types[3], 
       'Separated Bike Lane': osm_types[3],
       'Neighborhood Greenway / Bike Lane': osm_types[1], 
       'Buffered Bike Lanes': osm_types[2],
       'Raised Bike Lane': osm_types[3], 
}
coa['coa_osm_type'] = coa['coa_facilitytype'].map(coa_conversion)

In [5]:
# remove sharrows and nas
coa = coa[(coa['coa_osm_type'] != 'sharrow') & coa['coa_osm_type'].notna()]

# also remove multi-use paths for now
coa = coa[coa['coa_osm_type'] != 'multi use path']

In [6]:
coa['coa_osm_type'].value_counts(dropna=False)

coa_osm_type
bike lane             63
cycletrack            41
buffered bike lane    16
Name: count, dtype: int64

In [7]:
coa.head()

,coa_id,coa_facilitytype,coa_name,coa_builtby,coa_onroad,coa_bothsides,coa_source,coa_yearinstalled,coa_protection,coa_notes,geometry,coa_osm_type
4,5,Contraflow Bike Lane,Porter Pl NE,N/A,On,N,Cycle Atlanta Phase 1,NaN,Low,None,"LINESTRING (2229463.709 1369434.188, 2229490.5...",bike lane
11,12,Protected Bike Lane,Juniper St NE,N/A,On,Y,Cycle Atlanta Phase 1,NaN,High,None,"LINESTRING (2230749.844 1372324.236, 2230785.7...",cycletrack
20,21,Bike Lane,Murphy Ave SW,N/A,On,Y,Cycle Atlanta Phase 1,NaN,Low,None,"LINESTRING (2222329.538 1361034.107, 2222214.6...",bike lane
31,32,Buffered Bike Lane,Whitehall St SW,N/A,On,Y,Cycle Atlanta Phase 1,NaN,Medium,None,"LINESTRING (2222329.538 1361034.107, 2222340.8...",buffered bike lane
39,40,Two-Way Cycle Track,Courtland St SE/Gilmer St SE,N/A,None,None,Cycle Atlanta Phase 1,NaN,Unknown,None,"LINESTRING (2229772.685 1365781.111, 2230127.3...",cycletrack


In [8]:
#export
coa.to_file(config['bicycle_facilities_fp']/'network_modifications.gpkg',layer='coa')